`FCN.ipynb`

Create a fully connected neural network that determines the slope angle of phase data.

In [1]:
# ### GPU Check
# # check available GPU in terminal: watch -d -n 0.5 nvidia-smi
# # select one GPU to train on:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="2"

### Imports

In [2]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '/Users/andrew/Documents/Lab/aas/modules'))
import notebook_loading

from FC_Data import *
from FC_Network import FCN
from FC_Train import fc_train

importing Jupyter notebook from FC_Data.ipynb
importing Jupyter notebook from FC_Network.ipynb
importing Jupyter notebook from FC_Train.ipynb


### Setup Data

In [3]:
# extract the redundant baselines and their gains and data from miriad and calfits files
red_bls, gains, uvd = load_relevant_data('../zen_data/zen.2458098.58037.xx.HH.uv','../zen_data/zen.2458098.58037.xx.HH.uv.abs.calfits')

# seperate trining and testing redundant baselines 
# if we have not already done this, load them from disk
training_redundant_baselines_dict, testing_redundant_baselines_dict = get_or_gen_test_train_red_bls_dicts(red_bls, gains.keys())

# seperate the visiblites
training_baselines_data = get_seps_data(training_redundant_baselines_dict, uvd)
testing_baselines_data = get_seps_data(testing_redundant_baselines_dict, uvd)

### Create network

In [4]:
layer_nodes = [1024,768,512,256,128,64]
network = FCN(layer_nodes, learning_rate = 0.0001)
network.create_graph()

### Train

In [6]:
fc_train(network,
         5,
         50,
         16,
         'logs/test',
         5,
         (training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains,
         clean = True)